In [ ]:
import os
import fitz  # PyMuPDF
import json
from flask import Flask, request, url_for, render_template_string
from werkzeug.utils import secure_filename
from PIL import Image
from collections import defaultdict

# Configuration
UPLOAD_FOLDER = 'uploads'
IMAGE_FOLDER = 'static/images'
ALLOWED_EXTENSIONS = {'pdf'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(IMAGE_FOLDER, exist_ok=True)

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

PROCESSED_DATA = {}
current_file = None

PAGE_TEMPLATE = '''
<!doctype html>
<style>
  body { font-family: sans-serif; margin: 0; padding: 20px; display: flex; flex-direction: column; height: 100vh; }
  h1 { margin-bottom: 20px; }
  .section { background: #f0f0f0; padding: 10px; margin-bottom: 20px; border-radius: 5px; }
  #display-container { height: 400px; border: 1px solid #ccc; margin-bottom: 20px; display: flex; justify-content: center; align-items: center; flex-shrink: 0; }
  #display { font-size: 48px; text-align: center; max-height: 100%; max-width: 100%; }
  img { max-width: 100%; max-height: 100%; object-fit: contain; }
  .controls-section { display: flex; align-items: center; gap: 10px; flex-wrap: wrap; }
  .controls-section.align-left { justify-content: flex-start; }
  #fixed-context-display { font-size: 18px; line-height: 1.4em; white-space: pre-wrap; }
  .img-placeholder { font-style: italic; color: #888; }
  #footer { position: sticky; bottom: 0; background: #f0f0f0; padding: 10px; }
  #slider { width: 100%; }
  .title-bar {
  display: flex;
  justify-content: space-between;
  align-items: center;
  margin-bottom: 20px;
  }
  .header-bar {
    display: flex;
    align-items: center;
    gap: 10px;
  }
  .header-bar a {
    text-decoration: none;
    color: #333;
    font-weight: bold;
  }
  .header-bar img {
    height: 40px;
    width: auto;
  }
</style>
<div class="title-bar">
  <h1>Lector PDF RSVP</h1>
  <div class="header-bar">
    <a href="https://github.com/fithithub" target="_blank">fithithub</a>
    <img src="{{ url_for('static', filename='logos/fithithub logo.png') }}" alt="fithithub Logo">
  </div>
</div>
<div class="section controls-section align-left">
  <form method="post" enctype="multipart/form-data" style="display:flex; gap:10px; align-items:center;">
    <input type="file" name="file" accept="application/pdf">
    <button type="submit">Subir y Ver</button>
  </form>
</div>
{% if seq_len %}
<div class="section controls-section align-left">
  <button id="start">Iniciar</button>
  <button id="pause" disabled>Pausar</button>
</div>
<div class="section controls-section align-left">
  <label for="stepN">N:</label>
  <input type="number" id="stepN" value="20" style="width:60px;">
  <button id="backStep">-N</button>
  <button id="backOne">« Anterior</button>
  <button id="next">Siguiente »</button>
  <button id="fwdStep">+N</button>
</div>
<div class="section">
  <div class="controls-section align-left">
    <label>Velocidad Palabra (ms): <input type="number" id="speedInput" value="500" min="50" step="50" style="width:80px;"></label>
    <span id="wpmInfo"></span>
  </div>
  <div class="controls-section align-left">
    <label>Velocidad Imagen (ms): <input type="number" id="imageSpeedInput" value="2000" min="500" step="100" style="width:80px;"></label>
    <span id="ipmInfo"></span>
  </div>
  <div class="controls-section align-left">
    <span id="totalTime"></span>
    <span id="remainingTime" style="margin-left:20px;"></span>
  </div>
</div>
<div class="section">
  <div class="controls-section align-left">
    <label>Ir a frase:<br><input type="text" id="searchBox" placeholder="Buscar frase completa..." style="width:300px;"></label>
    <button id="searchBtn">Ir</button>
  </div>
  <div class="controls-section align-left">
    <label>Ir a posición:<br><input type="number" id="posInput" min="0" style="width:80px;"></label>
    <button id="posBtn">Ir</button>
  </div>
</div>
<div id="display-container"><div id="display">El texto se mostrará aquí</div></div>
<div class="section" id="fixed-context-display"></div>
<div id="footer" class="section">
  <div class="controls-section align-left">
    <span id="posInfo">Posición: 0 / {{ seq_len }} (0%)</span>
    <span id="remainInfo" style="margin-left:20px;"></span>
  </div>
  <input type="range" id="slider" min="0" max="{{ seq_len - 1 }}" value="0">
</div>
<script>
  const sequence = {{ seq_json | safe }};
  const total = sequence.length;
  const items = sequence.filter(item => item.type === 'word' || item.type === 'image');
  let index = 0, intervalId = null;

  const display = document.getElementById('display');
  const slider = document.getElementById('slider');
  const posInfo = document.getElementById('posInfo');
  const remainInfo = document.getElementById('remainInfo');
  const totalTimeEl = document.getElementById('totalTime');
  const remainingTimeEl = document.getElementById('remainingTime');
  const speedInput = document.getElementById('speedInput');
  const imageSpeedInput = document.getElementById('imageSpeedInput');
  const startBtn = document.getElementById('start');
  const pauseBtn = document.getElementById('pause');
  const backStepBtn = document.getElementById('backStep');
  const backOneBtn = document.getElementById('backOne');
  const nextBtn = document.getElementById('next');
  const fwdStepBtn = document.getElementById('fwdStep');
  const stepNInput = document.getElementById('stepN');
  const searchBox = document.getElementById('searchBox');
  const searchBtn = document.getElementById('searchBtn');
  const posInput = document.getElementById('posInput');
  const posBtn = document.getElementById('posBtn');
  const contextDisplay = document.getElementById('fixed-context-display');

  let speed = parseInt(speedInput.value, 10);
  let imageSpeed = parseInt(imageSpeedInput.value, 10);

  function formatDuration(sec) {
    const d = Math.floor(sec / 86400); sec %= 86400;
    const h = Math.floor(sec / 3600); sec %= 3600;
    const m = Math.floor(sec / 60);
    const s = sec % 60;
    return `${d}d ${h}h ${m}m ${s}s`;
  }

  function updateSpeedInfo() {
    document.getElementById('wpmInfo').textContent = ` (${Math.round(60000 / speed)} WPM)`;
    document.getElementById('ipmInfo').textContent = ` (${Math.round(60000 / imageSpeed)} IPM)`;
  }

  function updateTimeInfo() {
    let tTotal = 0;
    items.forEach(item => tTotal += (item.type === 'word' ? speed : imageSpeed));
    const secTotal = Math.round(tTotal / 1000);
    const elapsed = index > 0
      ? items.slice(0, index).reduce((acc, item) => acc + (item.type === 'word' ? speed : imageSpeed), 0)
      : 0;
    const secElapsed = Math.round(elapsed / 1000);
    totalTimeEl.textContent = `Total: ${formatDuration(secTotal)}`;
    remainingTimeEl.textContent = `Queda: ${formatDuration(secTotal - secElapsed)}`;
  }

  function updatePosInfo() {
    const pct = ((index / (items.length - 1)) * 100).toFixed(1);
    posInfo.textContent = `Posición: ${index} / ${items.length} (${pct}%)`;
    remainInfo.textContent = `Leído ${pct}% - Queda ${(100 - pct).toFixed(1)}%`;
  }

  function updateContextDisplay() {
    // Construir bloques de 20 elementos (palabra o imagen)
    const blocks = [];
    for (let i = 0; i < items.length; i += 20) {
        blocks.push(items.slice(i, i + 20));
    }
    const currentBlock = Math.floor(index / 20);
    const visible = [];
    for (let b = currentBlock - 2; b <= currentBlock + 2; b++) {
        if (b >= 0 && b < blocks.length) visible.push({ block: blocks[b], idx: b });
    }
    // Generar HTML solo con resaltado en bloque actual
    const html = visible.map(({ block, idx: blockIdx }) => {
        return block.map((item, i) => {
        const globalIdx = blockIdx * 20 + i;
        if (blockIdx === currentBlock && globalIdx === index) {
            return item.type === 'word'
            ? `<span style="background: yellow">${item.text}</span>`
            : `<span class="img-placeholder" style="background: yellow">[Imagen]</span>`;
        }
        // sin resaltado en otros bloques
        return item.type === 'word'
            ? item.text
            : `<span class="img-placeholder">[Imagen]</span>`;
        }).join(' ');
    }).join('<br>');
    contextDisplay.innerHTML = html;
    }


  function showItem(item) {
    if (item.type === 'word') display.textContent = item.text;
    else display.innerHTML = `<img src="${item.src}">`;
    slider.value = index;
    updatePosInfo(); updateTimeInfo(); updateSpeedInfo(); updateContextDisplay();
  }

  function step() {
    index = Math.max(0, Math.min(index, items.length - 1));
    showItem(items[index]);
  }

  function startInterval() {
    const current = items[index];
    const delay = current.type === 'image' ? imageSpeed : speed;
    intervalId = setTimeout(() => {
      const prev = current;
      index++;
      if (index < items.length) {
        // Si el anterior fue imagen y el siguiente es palabra, espera 1s
        if (prev.type === 'image' && items[index].type === 'word') {
          step();
          setTimeout(() => { startInterval(); }, 1000);
        } else {
          step();
          startInterval();
        }
      }
    }, delay);
  }

  startBtn.onclick = () => { if (intervalId) return; speed = parseInt(speedInput.value, 10); imageSpeed = parseInt(imageSpeedInput.value, 10); updateSpeedInfo(); step(); startBtn.disabled = true; pauseBtn.disabled = false; startInterval(); };
  pauseBtn.onclick = () => { clearTimeout(intervalId); intervalId = null; startBtn.disabled = false; pauseBtn.disabled = true; };
  backStepBtn.onclick = () => { index -= parseInt(stepNInput.value, 10); step(); };
  backOneBtn.onclick = () => { index--; step(); };
  nextBtn.onclick = () => { index++; step(); };
  fwdStepBtn.onclick = () => { index += parseInt(stepNInput.value, 10); step(); };
  slider.oninput = () => { index = parseInt(slider.value, 10); step(); };
  speedInput.onchange = imageSpeedInput.onchange = () => { clearTimeout(intervalId); intervalId = null; speed = parseInt(speedInput.value, 10); imageSpeed = parseInt(imageSpeedInput.value, 10); updateSpeedInfo(); step(); startInterval(); };
  searchBtn.onclick = () => { const phrase = searchBox.value.trim(); if (!phrase) return; const parts = phrase.split(/\s+/); let found = -1;
    let flat = items.map((el, i) => el.type === 'word'? el.text : '[Imagen]');
    for (let i = 0; i <= flat.length - parts.length; i++) {
      if (parts.every((w, j) => flat[i+j] === w)) { found = i; break; }
    }
    if (found >= 0) { index = found; step(); startBtn.click(); } else alert('Frase no encontrada'); };
  posBtn.onclick = () => { const p = parseInt(posInput.value, 10); if (!isNaN(p) && p >= 0 && p < items.length) { index = p; step(); startBtn.click(); } else alert('Posición inválida'); };
</script>
{% endif %}
'''

@app.route('/', methods=['GET','POST'])
def index():
    global current_file
    if request.method == 'POST':
        file = request.files.get('file')
        if file and allowed_file(file.filename):
            fname = secure_filename(file.filename)
            path = os.path.join(app.config['UPLOAD_FOLDER'], fname)
            file.save(path)
            seq = []
            doc = fitz.open(path)
            for pi, page in enumerate(doc):
                for img in page.get_images(full=True):
                    base = doc.extract_image(img[0])
                    ext = base['ext']
                    name = f"{pi}_{img[0]}.{ext}"
                    p = os.path.join(IMAGE_FOLDER, name)
                    with open(p, 'wb') as img_file:
                        img_file.write(base['image'])
                    img_obj = Image.open(p)
                    img_obj.thumbnail((800, 400), Image.ANTIALIAS)
                    img_obj.save(p)
                    seq.append({'type': 'image', 'src': url_for('static', filename=f"images/{name}")})
                lines_by_y = defaultdict(list)
                for w in page.get_text('words'):
                    x0, y0, x1, y1, text = w[:5]
                    y_key = round(y0 / 5) * 5
                    lines_by_y[y_key].append((x0, text))
                line_id = 0
                for y in sorted(lines_by_y):
                    for _, word in sorted(lines_by_y[y], key=lambda x: x[0]):
                        seq.append({'type': 'word', 'text': word, 'line': line_id})
                    line_id += 1
            PROCESSED_DATA['seq'] = seq
            current_file = 'seq'
    sequence = PROCESSED_DATA.get(current_file, [])
    return render_template_string(PAGE_TEMPLATE, seq_json=json.dumps(sequence), seq_len=len(sequence))

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)